In [ ]:
import numpy as np # linear algebra
import pandas as pd  # data processing
import os #  to interact with files using there paths
import cv2
from sklearn.datasets import load_files

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
data_dir =  r'drive/MyDrive/folder_image_Morocco_training_image_plants'


classes = os.listdir(data_dir) # Get names of classes
len(classes)

85

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

train_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory=data_dir,
                                                 shuffle=True,
                                                 target_size=(200, 200), 
                                                 subset="training",
                                                 class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory=data_dir,
                                                 shuffle=True,
                                                 target_size=(200, 200), 
                                                 subset="validation",
                                                 class_mode='categorical')

Found 18092 images belonging to 85 classes.
Found 4503 images belonging to 85 classes.


In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', input_shape=(200, 200,3), activation='relu', name='Conv2D_1'))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', name='Conv2D_2'))
model.add(MaxPool2D(pool_size=(2,2), name='Maxpool_1'))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu', name='Conv2D_3'))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', name='Conv2D_4'))
model.add(MaxPool2D(pool_size=(2,2), name='Maxpool_2'))
model.add(Dropout(0.25))
    
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu', name='Conv2D_5'))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu', name='Conv2D_6'))
model.add(MaxPool2D(pool_size=(2,2), name='Maxpool_3'))

model.add(Flatten())
model.add(Dense(units=512, activation='relu', name='Dense_1'))
model.add(Dropout(0.5))
#model.add(Dense(units=128, activation='relu', name='Dense_2'))
model.add(Dense(units=85, activation='softmax', name='Output'))

In [ ]:
# Getting the summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv2D_1 (Conv2D)           (None, 200, 200, 32)      896       
                                                                 
 Conv2D_2 (Conv2D)           (None, 198, 198, 32)      9248      
                                                                 
 Maxpool_1 (MaxPooling2D)    (None, 99, 99, 32)        0         
                                                                 
 dropout_3 (Dropout)         (None, 99, 99, 32)        0         
                                                                 
 Conv2D_3 (Conv2D)           (None, 99, 99, 64)        18496     
                                                                 
 Conv2D_4 (Conv2D)           (None, 97, 97, 64)        36928     
                                                                 
 Maxpool_2 (MaxPooling2D)    (None, 48, 48, 64)       

In [ ]:
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.optimizers as Optimizer
model.compile( optimizer=Optimizer.Adam(lr=0.0001),loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
batch_size=128
nb_epochs=100
model.fit(
    train_dataset,
    steps_per_epoch = train_dataset.samples // batch_size,
    validation_data = validation_dataset, 
    validation_steps = validation_dataset.samples // batch_size,
    epochs = nb_epochs)

Epoch 1/100
141/141 [==============================] - 994s 7s/step - loss: 4.4429 - accuracy: 0.0126 - val_loss: 4.4340 - val_accuracy: 0.0205
Epoch 2/100
141/141 [==============================] - 739s 5s/step - loss: 4.4000 - accuracy: 0.0225 - val_loss: 4.3580 - val_accuracy: 0.0375
Epoch 3/100
141/141 [==============================] - 557s 4s/step - loss: 4.2991 - accuracy: 0.0378 - val_loss: 4.2026 - val_accuracy: 0.0679
Epoch 4/100
141/141 [==============================] - 432s 3s/step - loss: 4.2042 - accuracy: 0.0534 - val_loss: 4.1585 - val_accuracy: 0.0714
Epoch 5/100
141/141 [==============================] - 324s 2s/step - loss: 4.1186 - accuracy: 0.0670 - val_loss: 4.0991 - val_accuracy: 0.0973
Epoch 6/100
141/141 [==============================] - 261s 2s/step - loss: 4.0034 - accuracy: 0.0786 - val_loss: 3.9271 - val_accuracy: 0.1089
Epoch 7/100
141/141 [==============================] - 192s 1s/step - loss: 3.9754 - accuracy: 0.0898 - val_loss: 3.9190 - val_accuracy:

In [ ]:
import tensorflow as tf

import tensorflow.keras.layers as L

import tensorflow.keras.backend as K
from tensorflow.keras.models import Model

from tensorflow.keras.applications import DenseNet121

import seaborn as sns
from tqdm import tqdm
import matplotlib.cm as cm
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tqdm.pandas()
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

np.random.seed(0)
tf.random.set_seed(0)

import warnings
warnings.filterwarnings("ignore")

In [ ]:
model = tf.keras.Sequential([DenseNet121(input_shape=(200, 200, 3),
                                             weights='imagenet',
                                             include_top=False),
                                 L.GlobalAveragePooling2D(),
                                 L.Dense(85,
                                         activation='softmax')])
        
model.compile(optimizer='adam',
                  loss = 'categorical_crossentropy',
                  metrics=['categorical_accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 6, 6, 1024)        7037504   
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense (Dense)               (None, 85)                87125     
                                                                 
Total params: 7,124,629
Trainable params: 7,040,981
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.optimizers as Optimizer
model.compile( optimizer=Optimizer.Adam(lr=0.0001),loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size=128
nb_epochs=100
model.fit(
    train_dataset,
    steps_per_epoch = train_dataset.samples // batch_size,
    validation_data = validation_dataset, 
    validation_steps = validation_dataset.samples // batch_size,
    epochs = nb_epochs)

Epoch 1/100
141/141 [==============================] - 67s 370ms/step - loss: 3.7680 - accuracy: 0.1873 - val_loss: 3.0579 - val_accuracy: 0.2991
Epoch 2/100
141/141 [==============================] - 50s 350ms/step - loss: 2.2673 - accuracy: 0.4989 - val_loss: 1.9789 - val_accuracy: 0.5402
Epoch 3/100
141/141 [==============================] - 50s 351ms/step - loss: 1.6072 - accuracy: 0.6292 - val_loss: 1.6355 - val_accuracy: 0.5964
Epoch 4/100
141/141 [==============================] - 50s 351ms/step - loss: 1.2560 - accuracy: 0.7134 - val_loss: 1.4991 - val_accuracy: 0.6196
Epoch 5/100
141/141 [==============================] - 50s 350ms/step - loss: 1.0455 - accuracy: 0.7553 - val_loss: 1.2921 - val_accuracy: 0.6732
Epoch 6/100
141/141 [==============================] - 49s 348ms/step - loss: 0.8899 - accuracy: 0.8036 - val_loss: 1.3324 - val_accuracy: 0.6696
Epoch 7/100
141/141 [==============================] - 52s 370ms/step - loss: 0.7398 - accuracy: 0.8366 - val_loss: 1.1547 -